In [1]:
%matplotlib inline
import math
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns; sns.set(style='ticks', color_codes=True)

from pandas.plotting import parallel_coordinates
from sklearn import preprocessing
from operator import itemgetter

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib

# pd.set_option('display.max_columns', 150)
pd.options.display.max_seq_items = 200

In [2]:
TRAIN_DATA = 'data/train.csv'
TEST_DATA = 'data/test.csv'
CLEAN_TRAIN_DATA = 'data/clean_train_data.csv'
NOISE_COLUMNS = ['Key', 'No', 'District', 'City', 'Hour', 'Minute', 'Date', 'Month', 'Year',
                'JunctionControl', 'dayInWeek', 'IsDayOrNight', 'CollisionDirection', 'DetailCause',
                'Tricycle']
ONE_HOT_COLUMNS = ['CollisionType', 'JunctionType']

In [3]:
#Normalization

def normalize_max_unknown(vector):
    min = np.min(vector)
    max = np.max(vector)
        
    return [(x - float(min)) / (float(max) - float(min)) for x in vector]

In [4]:
#Data Visualization

def parallel_lines(df, target_category):
    plt.pyplot.figure(figsize=(30,10))

    parallel_plt = parallel_coordinates(df, target_category)
#     plt.pyplot.savefig('%s_parallel.png' % target_category)
    
def scatter_plot(df, target_category):
    scatter_plt = sns.pairplot(df, hue=target_category)
    scatter_plt.savefig('%s_scatter.png' % target_category)

In [5]:
def generate_arr(dataset, classification):
    classification_arr = dataset[classification].values
    del dataset[classification]
    dataset_arr = dataset.values

    return dataset_arr, classification_arr

In [6]:
mmda_df = pd.read_csv(TRAIN_DATA)
mmda_df = mmda_df.drop(NOISE_COLUMNS, axis=1)

/home/anton/Dev/mmda-traffic-prediction/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
mmda_df_one_hot = pd.get_dummies(mmda_df, prefix=ONE_HOT_COLUMNS, columns=ONE_HOT_COLUMNS)

imputer = Imputer(strategy='median')
imputer.fit(mmda_df_one_hot)
imputer_result = imputer.transform(mmda_df_one_hot)

mmda_df_clean = pd.DataFrame(imputer_result, columns=mmda_df_one_hot.columns)

mmda_df_clean.to_csv(CLEAN_TRAIN_DATA)

In [8]:
mmda_df_clean = pd.read_csv(CLEAN_TRAIN_DATA)
# corr_matrix = mmda_df_clean.corr()
# corr_matrix['Classification_1'].sort_values(ascending=False)

In [9]:
mmda_arr, classification_arr = generate_arr(mmda_df_clean, 'Classification')
# labels = [[x] for x in classification_arr]
# binary_labels = MultiLabelBinarizer().fit_transform(labels)
# binary_labels

In [10]:
clf = GaussianNB()
clf.fit(mmda_arr, classification_arr)
joblib.dump(clf, 'mmda_predictor.pkl')

NameError: name 'joblib' is not defined